In [1]:
import py_entitymatching as em
import pandas as pd
import os
import sys
import numpy as np

In [2]:
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]
pandas version: 2.2.2
magellan version: 0.4.0


In [3]:
file = 'stadiums_final_dirty_fuzzy_fd.csv'
path_A = 'em_inputs' + os.sep + file
path_B = 'em_inputs' + os.sep + file

In [4]:
A = em.read_csv_metadata(path_A, key = "index")
B = em.read_csv_metadata(path_B, key = "index")

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 105
Number of tuples in B: 105
Number of tuples in A X B (i.e the cartesian product): 11025


In [6]:
# Initialize overlap blocker
pd.options.mode.chained_assignment = None  # default='warn'
ob = em.OverlapBlocker()
# Block over title attribute
#attrs = ['name','capacity','location','opened','team']
#attrs = ['player', 'position', 'team', 'facility', 'location', 'capacity', 'opened']
attrs = ['player', 'team', 'facility']

C = ob.block_tables(A, B, 'facility', 'facility', 
                    l_output_attrs=attrs,
                    r_output_attrs=attrs, 
                    show_progress=False, overlap_size=2)
len(C)

603

In [7]:
S = em.sample_table(C, 5)


In [8]:
#G = em.label_table(S, label_column_name='gold_labels')

In [9]:
match_f = em.get_features_for_matching(A, B, validate_inferred_attr_types = False)

In [10]:
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,index_index_exm,index,index,None,None,exact_match,<function index_index_exm at 0x7f86042696c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,index_index_anm,index,index,None,None,abs_norm,<function index_index_anm at 0x7f8604269630>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,index_index_lev_dist,index,index,None,None,lev_dist,<function index_index_lev_dist at 0x7f8604269900>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,index_index_lev_sim,index,index,None,None,lev_sim,<function index_index_lev_sim at 0x7f8499bb1000>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,player_player_jac_qgm_3_qgm_3,player,player,qgm_3,qgm_3,jaccard,<function player_player_jac_qgm_3_qgm_3 at 0x7f8499bb0c10>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,player_player_cos_dlm_dc0_dlm_dc0,player,player,dlm_dc0,dlm_dc0,cosine,<function player_player_cos_dlm_dc0_dlm_dc0 at 0x7f8499bb0ca0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,player_player_jac_dlm_dc0_dlm_dc0,player,player,dlm_dc0,dlm_dc0,jaccard,<function player_player_jac_dlm_dc0_dlm_dc0 at 0x7f8499bb12d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,player_player_mel,player,player,None,None,monge_elkan,<function player_player_mel at 0x7f8499bb1360>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,player_player_lev_dist,player,player,None,None,lev_dist,<function player_player_lev_dist at 0x7f8499bb13f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,player_player_lev_sim,player,player,None,None,lev_sim,<function player_player_lev_sim at 0x7f8499bb1480>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [11]:
brm = em.BooleanRuleMatcher()
rule_1 = ['player_player_jac_dlm_dc0_dlm_dc0(ltuple, rtuple) > 0.3', 'player_player_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) > 0.3', 'player_player_lev_sim(ltuple, rtuple) > 0.3']
#rule_2 = ['facility_facility_jac_dlm_dc0_dlm_dc0(ltuple, rtuple) > 0.8', 'facility_facility_cos_dlm_dc0_dlm_dc0(ltuple, rtuple)> 0.8', 'facility_facility_lev_sim(ltuple, rtuple) > 0.8']

In [12]:

rule_name = brm.add_rule(rule_1, match_f)
#rule_name = brm.add_rule(rule_2, match_f)

In [13]:
predictions = brm.predict(table=C, target_attr='predicted_labels', inplace=True)
C['predictions'] = predictions

In [14]:

CC  = C[C['ltable_index'] != C['rtable_index']]
CCC = CC[['ltable_index', 'rtable_index', 'predictions']]
final_cols = list(CCC.columns)
final_rows = []
indexed_rows = set()
for index, rows in CCC.iterrows():
    ltable = rows['ltable_index']
    rtable = rows['rtable_index']
    predictions = rows['predictions']
    if (ltable, rtable) not in indexed_rows and (rtable, ltable) not in indexed_rows:
        final_rows.append((ltable, rtable, predictions))
        indexed_rows.add((ltable, rtable))
final_dataframe = pd.DataFrame(final_rows, columns= final_cols )     
#CCC = pd.DataFrame(np.sort(CCC.values, axis=1), columns=CCC.columns).drop_duplicates(ignore_index = True)

In [15]:
final_dataframe.to_csv("em_outputs/em_result_"+file, index = False)

In [16]:
final_dataframe

,ltable_index,rtable_index,predictions
0,20,0,0
1,23,0,0
2,37,0,0
3,45,0,0
4,47,0,0
...,...,...,...
245,103,101,0
246,104,101,0
247,103,102,0
248,104,102,0


In [17]:
remove_tuples = set()

for index, row in final_dataframe.iterrows():
    if row['predictions'] == 1:
        remove_tuples.add(min(row['ltable_index'], row['rtable_index']))

In [18]:
ltable_cleaned = A[~A.index.isin(remove_tuples)]

In [19]:
ltable_cleaned

,index,player,position,team,facility,location,capacity,opened
0,0,justin herbert,qb,chargers,sofi stadium,"inglewood, california","70,240",2020
1,1,david montgomery,rb,bears,soldier field,"chicago, illinois","61,500",1924[nb 1]
2,2,von miller,lb,bills,bills stadium,"orchard park, new york","71,608",1973
3,3,trent williams,ot,49ers,levi's stadium,"santa clara, california","68,500",2014
4,4,lamar jackson,qb,ravens,m&t bank stadium,"baltimore, maryland","71,008",1998
...,...,...,...,...,...,...,...,...
100,100,mike evans,wr,buccaneers,raymond james stadium,NaN,"65,890",1998
101,101,quenton nelson,g,colts,lucas stadium,NaN,"67,000",NaN
102,102,shaquille leonard,lb,colts,lucas stadium,NaN,"67,000",NaN
103,103,kenny moore ii,cb,colts,lucas stadium,NaN,"67,000",NaN


In [20]:
A

,index,player,position,team,facility,location,capacity,opened
0,0,justin herbert,qb,chargers,sofi stadium,"inglewood, california","70,240",2020
1,1,david montgomery,rb,bears,soldier field,"chicago, illinois","61,500",1924[nb 1]
2,2,von miller,lb,bills,bills stadium,"orchard park, new york","71,608",1973
3,3,trent williams,ot,49ers,levi's stadium,"santa clara, california","68,500",2014
4,4,lamar jackson,qb,ravens,m&t bank stadium,"baltimore, maryland","71,008",1998
...,...,...,...,...,...,...,...,...
100,100,mike evans,wr,buccaneers,raymond james stadium,NaN,"65,890",1998
101,101,quenton nelson,g,colts,lucas stadium,NaN,"67,000",NaN
102,102,shaquille leonard,lb,colts,lucas stadium,NaN,"67,000",NaN
103,103,kenny moore ii,cb,colts,lucas stadium,NaN,"67,000",NaN


Prepare groundtruth for comparison. Since the partitioned tables may not have complete information, we only include those columns for the evaluation whose information are available (the columns participating on joins.)

In [21]:
gt_table = pd.read_csv(r"em_gold/em_stadium_gold_complete_dirty.csv")
dirty_groundtruth = set()
for index, row in gt_table.iterrows():
    player = row['Player'].lower()
    team = row['Team'].lower()
    facility = row['Facility'].lower()
    dirty_groundtruth.add((player, team, facility))
print("the dirty groundtruth size is:", len(dirty_groundtruth))

gt_table = pd.read_csv(r"em_gold/em_stadium_gold_complete.csv")
clean_groundtruth = set()
for index, row in gt_table.iterrows():
    player = row['Player'].lower()
    team = row['Team'].lower()
    facility = row['Facility'].lower()
    clean_groundtruth.add((player, team, facility))
print("the clean groundtruth size is:", len(clean_groundtruth))

the dirty groundtruth size is: 106
the clean groundtruth size is: 93


Prepare integrated table for comparison for both dirty (A) and clean (ltable_cleaned) tables.

In [22]:
integration_result_dirty = set()
for index, row in A.iterrows():
    player = row['player']
    team = row['team']
    facility = row['facility']
    integration_result_dirty.add((player, team, facility))
print("The dirty result size is:", len(integration_result_dirty))

integration_result_clean = set()
for index, row in ltable_cleaned.iterrows():
    player = row['player']
    team = row['team']
    facility = row['facility']
    integration_result_clean.add((player, team, facility))
print("The clean result size is:", len(integration_result_clean))

The dirty result size is: 105
The clean result size is: 92


Print results out

In [23]:
print("Dirty table size |T|: ", A.shape[0])
print("Clean table size |T|: ", ltable_cleaned.shape[0])
#print("Dirty table intersection with dirty ground truth |T int T*|: ", len(clean_groundtruth.intersection(integration_result_dirty)))
print("Clean table intersection with clean ground truth |T int T*|: ", len(clean_groundtruth.intersection(integration_result_clean)))


Dirty table size |T|:  105
Clean table size |T|:  92
Clean table intersection with clean ground truth |T int T*|:  79


In [24]:
precision = len(clean_groundtruth.intersection(integration_result_clean))/ len(integration_result_clean)
recall = len(clean_groundtruth.intersection(integration_result_clean)) / len(clean_groundtruth)
f1_score = (2 * precision * recall) / (precision + recall)
print("Precision = ", precision)
print("Recall = ", recall)
print ("F1-score = ", f1_score)

Precision =  0.8586956521739131
Recall =  0.8494623655913979
F1-score =  0.854054054054054
